# Modeling

## [1. Importing Packages and Methods](#1_importing_packages_and_methods)
## [2. Importing Data and Labeling](#2_importing_data_and_labeling)
## [3. Trading Bots Creation](#3_trading_bots_creations)
- ### [3.1 Conditional Trading](#3_1_conditional_trading)
    - #### [3.1.1 Conditional Trading Momentum - SMA](#3_1_1_conditional_trading_momentum_sma)
    - #### [3.1.2 Conditional Bollinger Momentum](#3_1_2_conditional_bollinger_momentum)
    - #### [3.1.3 Conditional Bollinger Contrarian](#3_1_3_conditional_bollinger_contrarian)
- ### [3.2 Exponential Moving Average](#3_2_exponential_moving_average)
    - #### [3.2.1 Exponential Momentum](#3_2_1_exponential_momentum)
- ### [3.3 Double Rolling](#3_3_double_rolling)
    - #### [3.3.1 SMA on EMA](#3_3_1_sma_on_ema)
- ### [3.4 Heikin Ashi and Bollinger](#3_4_heikin_ashi_and_bollinger)
    - #### [3.4.1 Heikin Ashi and Bollinger Momentum](#3_4_1_heikin_ashi_and_bollinger_momentum)
- ### [QQ Graph](#qq_graph)


## 1. Importing Packages and Methods<a id='1_importing_packages_and_methods'></a>

In [2]:
import os
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


## Import desired packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import statsmodels.api as sm
import gc

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from tqdm import tqdm
import itertools
import gc


## 2. Importing Data and Labeling<a id='2_importing_data_and_labeling'></a>

In [3]:
# Load images and labels


def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image = Image.open(os.path.join(folder_path, filename)).resize((224, 224))
            images.append(np.array(image))
            labels.append(label)
    return images, labels

folder_0 = "/Users/liqingyang/Documents/GitHub/head_shoulder_bots/data/processed_data/non_hs"
folder_1 = "/Users/liqingyang/Documents/GitHub/head_shoulder_bots/data/processed_data/hs"

images_0, labels_0 = load_images(folder_0, 0)
images_1, labels_1 = load_images(folder_1, 1)

images = np.array(images_0 + images_1)
labels = np.array(labels_0 + labels_1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rescale=1./255
)
datagen.fit(X_train)

# Define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test/255, y_test), epochs=10)

KeyboardInterrupt: 